# Created by Faisal
# 21/7/2019
# DQN algorithm on the cart and pole problem using a neural network built via Tensorflow
# The algorithm is a simpler version of the DQN in 'Playing Atari with Deep Reinforcement Learning' paper by Volodymyr et al and with out applying Expericne replay

In [1]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand

## Build the Q network

In [2]:
n_inputs = 4
n_hidden = 4
n_output = 2

### Network placeholders

In [3]:
X = tf.placeholder(tf.float32,(None,n_inputs),name='X')
y = tf.placeholder(tf.float32,(None),name='y')

### Hidden layer

In [4]:
w_hidden = tf.Variable(tf.random_normal((n_inputs, n_hidden)), name='w_hidden')
b_hidden = tf.Variable(tf.random_normal((n_hidden, )), name='b_hidden')
x_hidden = tf.nn.relu(tf.matmul(X,w_hidden) + b_hidden)

### output layer is a 2 dimentional vector with linear activation, each index is the Q(s,a) for a specific action in this problem we have two discrete actions, 0 which is apply a force to the left, and 1 which apply a force to the right

In [5]:
w_output = tf.Variable(tf.random_normal((n_hidden, n_output)), name='w_output')
b_output = tf.Variable(tf.random_normal((n_output,)), name='b_hidden')
output = tf.matmul(x_hidden,w_output) + b_output

In [6]:
with tf.name_scope('loss'):
    loss = tf.losses.mean_squared_error(y,output)

In [7]:
learning_rate = 0.001
with tf.name_scope('optim'):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
with tf.name_scope('summaries'):
    tf.summary.scalar('loss',loss)
    merged = tf.summary.merge_all()

In [67]:
num_eps = 50000
# discount factor
Gamma = 0.95
Actions = [0,1]

In [ ]:
env = gym.make('CartPole-v0')
rewards = []
r = 0
with tf.Session() as sess:
    init.run()
    for i in range(num_eps):
        epsilon = 0.1
        state = env.reset()
        r = 0
        done = False
        while True:
            # Choose actions according to an eps-greedy policy
            if rand() <= epsilon:
                # compute the Q for each action
                Q = sess.run(output,feed_dict={X:state.reshape(1,4)})
                # choose the suboptimal action
                action = 1 - np.argmax(Q)
            else:
                Q = sess.run(output,feed_dict={X:state.reshape(1,4)})
                # choose actions greedly
                action = np.argmax(Q)
            action = np.random.choice([0,1])
            state_,reward,done,_ = env.step(action)
            if done:
                yi = reward
                r += reward
                rewards.append(r)
                print ('\rreward: {}, iteration: {} , max_rewrad: {}, avergae_rewrad {:.2f}, eps: {} \r'.format(r, i+1, 
                                                            np.max(rewards),np.mean(rewards),epsilon),end='')
                sess.run([train_op,loss],feed_dict={X:state.reshape(1,4),y:yi})
                break
            Q_ = sess.run(output,feed_dict={X:state_.reshape(1,4)})
            r += reward
            # compute a Q learning return which is will be the network target
            yi = reward + (Gamma * np.max(Q_))
            sess.run(train_op,feed_dict={X:state.reshape(1,4),y:yi})
            l = sess.run(loss,feed_dict={X:state.reshape(1,4),y:yi})
            state = state_
        # save every five episodes
        if (i%5 == 0):
            saver.save(sess,'./DQN_for_cart_and_pole.ckpt')

In [66]:
# Run the policy after Training
re = 0
obs = env.reset()
with tf.Session() as sess:
    saver.restore(sess,'./DQN_for_cart_and_pole.ckpt')
    while True:
        env.render()
        actions = sess.run(output,feed_dict={X:obs.reshape(1,4)})
        action = np.argmax(actions)
        obs,r,d,_ = env.step(action)
        re += r
        if d:
            env.close()
            break
re        

INFO:tensorflow:Restoring parameters from ./DQN_for_cart_and_pole.ckpt


110.0